# Visualizations in Plotly for the DMP Aspect Test Experiment

In [ ]:
%matplotlib notebook

In [ ]:
import sys
sys.path.append("../../")

from dmp.data.logging import _get_sql_engine
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
from joblib import Memory
from ipywidgets import interact, interact_manual

import plotly.graph_objects as go

In [ ]:
db = _get_sql_engine()

memory = Memory(location="./cache", verbose=0)

def query(query_string):
    with db.connect() as engine:
        return pd.read_sql(query_string, engine)

cached_query = memory.cache(query)

def clear_cache():
    memory.clear()
    
options = {
    "dataset": ['529_pollen',
        'sleep',
        'adult',
        '537_houses',
        'nursery',
        '201_pol',
        'mnist',
        'connect_4',
        'wine_quality_white'],
    "agg": ["avg", "min", "max"],
    "topology" : ["exponential", "rectangle", "trapezoid", "wide_first"],
    "loss": ['history_loss', 'history_hinge', 'history_accuracy',
       'history_val_loss', 'history_val_hinge', 'history_val_accuracy',
       'history_squared_hinge', 'history_cosine_similarity',
       'history_val_squared_hinge', 'history_mean_squared_error',
       'history_mean_absolute_error', 'history_val_cosine_similarity',
       'history_val_mean_squared_error', 'history_root_mean_squared_error',
       'history_val_mean_absolute_error',
       'history_kullback_leibler_divergence',
       'history_val_root_mean_squared_error',
       'history_mean_squared_logarithmic_error',
       'history_val_kullback_leibler_divergence',
       'history_val_mean_squared_logarithmic_error'],
    "residual_mode": ["none", "full"]
}

In [ ]:
clear_cache()

# 3D Heatmap App

In [ ]:
@interact_manual(**options, viz=["imshow", "scatter"])
def heatmap_app_3d(groups="('exp00', 'exp01')", dataset="537_houses", topology="wide_first", loss="history_val_mean_squared_error", agg="avg", residual_mode="none", viz="imshow"):
    query_string = f'''
    select "config.budget", "config.depth", {agg}(a.val) as value, count(a.val), a.epoch
    from
        materialized_experiments_0 t,
        unnest(t.{loss}) WITH ORDINALITY as a(val, epoch)
    WHERE
        "groupname" IN {groups} and
        "config.dataset"='{dataset}' and
        "config.topology"='{topology}' and
        "config.residual_mode"='{residual_mode}'
    GROUP BY epoch, "config.budget", "config.depth"
    '''
    df = cached_query(query_string)#.query("count == 30")
    
    if viz=="scatter":
        
        fig = px.scatter_3d(df,
                    x='config.depth',
                    y='config.budget',
                    z='epoch',
                    color='value',
                    log_y=True,
                    opacity=0.25)
        fig.show()
        
    elif viz=="imshow":

        dimensions = ["config.budget", "config.depth", "epoch"]

        df[dimensions[0]] = df[dimensions[0]].astype(int)
        df[dimensions[1]] = df[dimensions[1]].astype(int)
        df[dimensions[2]] = df[dimensions[2]].astype(int)

        x_labels, y_labels, z_labels = [sorted(df[dim].unique()) for dim in dimensions]
        X, Y, Z = np.mgrid[0:len(x_labels), 0:len(y_labels), 0:len(z_labels)]
        values = np.empty((X+Y+Z).shape)
        values[:] = np.NaN

        x_idx = {a:b for b,a in enumerate(x_labels)}
        y_idx = {a:b for b,a in enumerate(y_labels)}
        z_idx = {a:b for b,a in enumerate(z_labels)}

        for _, row in df.iterrows():
            values[x_idx[row[dimensions[0]]], y_idx[row[dimensions[1]]], z_idx[row[dimensions[2]]]] = row["value"]


        fig = go.Figure(data=go.Volume(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=values.flatten(),
            opacity=0.1, # needs to be small to see through all surfaces
            surface_count=30
            ))

        fig.update_layout(scene = dict(
                            xaxis = dict(ticktext=x_labels,
                                         tickvals=list(range(0,len(x_labels))),
                                         title=dimensions[0]),

                            yaxis = dict(ticktext=y_labels,
                                         tickvals=list(range(0,len(y_labels))),
                                         title=dimensions[1]),

                            zaxis = dict(ticktext=z_labels,
                                         tickvals=list(range(0,len(z_labels))),
                                         title=dimensions[2]),


                            ),
                          width=700, height=700,
                         title=f"{loss} for {dataset}, {topology}, residual {residual_mode}")
        fig.show()



# 2D Heatmap App

In [ ]:
### Heapmap App using Imshow

@interact_manual(**options, viz=["imshow", "scatter"])
def heatmap_app(groups="('exp00', 'exp01')", dataset="537_houses", topology="wide_first", loss="history_val_mean_squared_error", agg="avg", residual_mode="none", viz="imshow"):
    query_string = f'''
select "config.budget", {agg}(a.val) as value, count(a.val), a.epoch
from
    materialized_experiments_0 t,
    unnest(t.{loss}) WITH ORDINALITY as a(val, epoch)
WHERE
    "groupname" IN {groups} and
    "config.dataset"='{dataset}' and
    "config.topology"='{topology}' and
    "config.residual_mode"='{residual_mode}'
GROUP BY epoch, "config.budget"
'''
    df = cached_query(query_string)
    
    if viz == "imshow":
        #df = df.query("count > 389")
        df["value"] = np.log(df["value"])
        img = df.pivot_table(columns="epoch", index="config.budget", values="value")
        img.index = img.index.astype("str") #plotly interprets the y axis as numeric otherwise
        return px.imshow(img)
    elif viz == "scatter":
        df = df.sort_values(["epoch", "config.budget"], ascending=[True, False])
        df["config.budget"] = df["config.budget"].astype("str")
        df["epoch"] = df["epoch"].astype("str")
        return px.scatter(df, x="epoch", y="config.budget", size="count", color="value")

## Choose Your Axis Heatmap Plot

In [ ]:
### Heapmap App using Imshow

axis = ["dataset", "topology", "residual_mode", "budget"]

@interact_manual(**options, viz=["imshow", "scatter"], y_dim=axis)
def heatmap_app(groups="('exp00', 'exp01')", x_dim="epoch", y_dim="budget", budget="64", dataset="537_houses", topology="wide_first", loss="history_val_mean_squared_error", agg="avg", residual_mode="none", viz="imshow"):
    axis = ["dataset", "topology", "residual_mode", "budget"]
    args = locals()
    filter_string = " and ".join([f'''"config.{s}"='{args[s]}' ''' for s in axis if s != y_dim])
    query_string = f'''
select "config.{y_dim}", {agg}(a.val) as value, count(a.val), a.epoch
from
    materialized_experiments_0 t,
    unnest(t.{loss}) WITH ORDINALITY as a(val, epoch)
WHERE
    "groupname" IN {groups} and
    {filter_string}
GROUP BY {x_dim}, "config.{y_dim}"
'''
    df = cached_query(query_string)
    
    if viz == "imshow":
        #df = df.query("count > 389")
        img = df.pivot_table(columns=x_dim, index="config."+y_dim, values="value")
        img.index = img.index.astype("str") #plotly interprets the y axis as numeric otherwise
        return px.imshow(img)
    elif viz == "scatter":
        df = df.sort_values(["epoch", "config.budget"], ascending=[True, False])
        df["config.budget"] = df["config.budget"].astype("str")
        df["epoch"] = df["epoch"].astype("str")
        return px.scatter(df, x="epoch", y="config.budget", size="count", color="value")

# Line Plot for Single budget x dataset x topology

In [ ]:
### single run app

@interact(**options)
def heatmap_app(dataset="537_houses", topology="wide_first", loss="history_val_mean_squared_error", agg="avg", residual_mode="none",
               budget="256"):
    query_string = f'''
select a.val as value, a.epoch, run_name
from
    materialized_experiments_0 t,
    unnest(t.{loss}) WITH ORDINALITY as a(val, epoch)
WHERE
    "groupname" IN ('exp00', 'exp01') and
    "config.dataset"='{dataset}' and
    "config.topology"='{topology}' and
    "config.residual_mode"='{residual_mode}' and
    "config.budget"='{budget}'
'''
    df = cached_query(query_string)
    return px.line(df, x="epoch", y="value", color="run_name")

##

- 537 houses, (val) kl divergence, trapezoid
- sleep, trapezoid, history_val_loss, 16777216

Notes:
- Logarithmic color map
- look into why sleep runs crash for high budgets
- plateau effect
- review stopping configurations / what is reasonable number of iterations?

# Next Steps

## Look at "plateau" feature from Charle's notebooks
## Add dataset, dataset complexity, as axis
## Add topology as an axis
### learning rate / optimizers / other axis to add

# To re-run using fixed epoch length, how long is reasonable?

In [ ]:
### Heapmap App using Imshow

@interact_manual
def heatmap_app(groups="('exp00', 'exp01')"):
    query_string = f'''
select
    "config.budget",
    "config.dataset",
    avg(array_length(t.history_loss, 1)) as avg,
    min(array_length(t.history_loss, 1)) as min,
    max(array_length(t.history_loss, 1)) as max,
    (percentile_disc(0.90) within group (order by array_length(t.history_loss, 1))) as pctile_90,
    stddev(array_length(t.history_loss, 1)) as std
from
    materialized_experiments_0 t
WHERE
    "groupname" IN {groups}
GROUP BY "config.budget", "config.dataset"
'''
    df = cached_query(query_string).sort_values("config.budget")
    
    df["spread"] = (df["max"] - df["min"])/2
    
    fig = px.line(df, x="config.budget", y="val", color="config.dataset", log_x=True, log_y=True)
    
    budgets = np.array(sorted(df["config.budget"].unique()))
    #fig.add_trace(go.Line(y=1/budgets, x=budgets, name="1/budget", line=dict(color='red', width=4, dash='dash') ))  
    #fig.add_trace(go.Line(y=5000/budgets, x=budgets, name="5000 * 1/budget", line=dict(color='black', width=4, dash='dash') ))
    return fig

In [ ]:
### Heapmap App using Imshow

@interact_manual
def heatmap_app(groups="('exp00', 'exp01')"):
    query_string = f'''
select
    "config.budget",
    "config.dataset",
    avg(array_length(t.history_loss, 1)) as avg,
    min(array_length(t.history_loss, 1)) as min,
    max(array_length(t.history_loss, 1)) as max,
    (percentile_disc(0.90) within group (order by array_length(t.history_loss, 1))) as pctile_90,
    stddev(array_length(t.history_loss, 1)) as std
from
    materialized_experiments_0 t
WHERE
    "groupname" IN {groups}
GROUP BY "config.budget", "config.dataset"
'''
    df = cached_query(query_string).sort_values("config.budget")
    
    df["pctile_90*budget"] = df["pctile_90"]*df["config.budget"]
    
    fig = px.line(df, x="pctile_90*budget", y="pctile_90", color="config.dataset", log_x=True, log_y=True)
    
    budgets = np.array(sorted(df["config.budget"].unique()))
    #fig.add_trace(go.Line(y=1/budgets, x=budgets, name="1/budget", line=dict(color='red', width=4, dash='dash') ))  
    #fig.add_trace(go.Line(y=5000/budgets, x=budgets, name="5000 * 1/budget", line=dict(color='black', width=4, dash='dash') ))
    return fig

In [ ]:
### Epoch Count App

@interact_manual(**options, viz=["imshow", "scatter"])
def epoch_count_app(groups="('exp00', 'exp01')", dataset="537_houses", topology="wide_first", residual_mode="none", viz="imshow"):
    query_string = f'''
select
    "config.budget",
    array_length(t.history_loss, 1) as num_epochs
from
    materialized_experiments_0 t
WHERE
    "groupname" IN {groups} and
    "config.dataset"='{dataset}' and
    "config.topology"='{topology}' and
    "config.residual_mode"='{residual_mode}'
'''
    df = cached_query(query_string)
    df = df.sort_values("config.budget")
    df["config.budget"] = df["config.budget"].astype("str")
    
    return px.scatter(df, x="config.budget", y="num_epochs", title=f"Number of epochs for {dataset}, {topology} in {groups}")

In [ ]:
### Epoch Count App

@interact_manual(**options, viz=["imshow", "scatter"])
def epoch_count_app(groups="('exp00', 'exp01')", dataset="537_houses", topology="wide_first", residual_mode="none", viz="imshow"):
    query_string = f'''
select
    "config.budget",
    "config.topology",
    array_length(t.history_loss, 1) as num_epochs
from
    materialized_experiments_0 t
WHERE
    "groupname" IN {groups} and
    "config.dataset"='{dataset}' and
    "config.residual_mode"='{residual_mode}'
'''
    df = cached_query(query_string).sort_values("config.budget")
    
    df["config.budget"] = df["config.budget"].astype("str")
    
    return px.violin(df,
                    x="config.budget",
                    y="num_epochs",
                    color="config.topology",
                     box=True,
                    title=f"Number of epochs for {dataset}, in {groups}")

In [ ]:
### Epoch Count App

@interact_manual(**options, viz=["imshow", "scatter"])
def epoch_count_agg_app(groups="('exp00', 'exp01')", dataset="537_houses", topology="wide_first", residual_mode="none", viz="imshow"):
    query_string = f'''
select
    "config.budget",
    "config.dataset",
    array_length(t.history_loss, 1) as num_epochs
from
    materialized_experiments_0 t
WHERE
    "groupname" IN {groups} and
    "config.topology"='{topology}' and
    "config.residual_mode"='{residual_mode}'
'''
    df = cached_query(query_string).sort_values("config.budget")
    
    df["config.budget"] = df["config.budget"].astype("str")
    
    return px.violin(df,
                    x="config.budget",
                    y="num_epochs",
                    color="config.dataset",
                     box=True,
                    title=f"Number of epochs for {topology}, in {groups}")

In [ ]:
### Generate 90th percentile epoch length for each experiment
query_string = f'''
select
    (percentile_disc(0.9) within group (order by array_length(t.history_loss, 1))) as epoch_90_pctile,
    "config.topology",
    "config.residual_mode",
    "config.dataset",
    "config.budget"
FROM
    materialized_experiments_0 t
WHERE
    "groupname" IN ('exp00','exp01')
GROUP BY
    "config.topology",
    "config.residual_mode",
    "config.dataset",
    "config.budget"
'''

df = cached_query(query_string)

df

In [ ]:
### Generate 90th percentile epoch length for each experiment
query_string = f'''
    select
        max(array_length(t.history_loss, 1)) as epoch_max,
        (percentile_disc(0.90) within group (order by array_length(t.history_loss, 1))) as epoch_90_pctile,
        "config.topology",
        "config.residual_mode",
        "config.dataset",
        "config.budget"
    FROM
        materialized_experiments_0 t
    WHERE
        "groupname" IN ('exp00','exp01')
    GROUP BY
        "config.topology",
        "config.residual_mode",
        "config.dataset",
        "config.budget"
'''

df = cached_query(query_string)

df

In [ ]:
df2 = df.groupby(["config.topology", "config.residual_mode", "config.dataset"]).max().drop("config.budget", axis=1)

df2.to_csv("epoch_90th_pctile.csv")
df2

In [ ]:
df.to_csv("epoch_90th_pctile_each_budget.csv")

In [ ]:
## Now, we want to add dataset (or some statistic derived from the dataset's complexity) and topology as axis.

In [ ]:
df = cached_query('''select * from materialized_experiments_0 WHERE "groupname" IN ('exp04') limit 10''')
display(df.columns)
df

In [ ]:
# Plot the heatmap of validation loss for one topology, one dataset, over iteration count and different budgets

q = '''
select "config.budget", avg(a.val) as val_loss, avg(a.loss) as train_loss, count(a.loss), a.epoch
from
    materialized_experiments_0 t,
    unnest(t.history_val_mean_squared_error, t.history_mean_squared_error) WITH ORDINALITY as a(val, loss, epoch)
WHERE
    "groupname" IN ('exp00', 'exp01') and
    "config.dataset"='537_houses' and
    "config.topology"='wide_first' and
    "config.residual_mode"='none'
GROUP BY epoch, "config.budget"
'''

# q = '''
# select distinct("name")
# from
#     materialized_experiments_0
# WHERE
#     "groupname" IN ('exp00', 'exp01') and
#     "config.dataset"='537_houses' and
#     "config.topology"='wide_first' and
#     "config.residual_mode"='none' and
#     "config.budget"=262144
# limit 10
# '''

df = cache_query(q)
df

In [ ]:
# Plot the heatmap of validation loss for one topology, one dataset, over iteration count and different budgets

query_string = f'''
select
    "config.dataset",
    avg(ln(array_length(t.history_loss, 1))) as log_avg_epoch
from
    materialized_experiments_0 t
WHERE
    "groupname" IN ('exp00', 'exp01')
GROUP BY "config.dataset"
'''

#(percentile_disc(0.90) within group (order by array_length(t.history_loss, 1))) as val

# q = '''
# select distinct("name")
# from
#     materialized_experiments_0
# WHERE
#     "groupname" IN ('exp00', 'exp01') and
#     "config.dataset"='537_houses' and
#     "config.topology"='wide_first' and
#     "config.residual_mode"='none' and
#     "config.budget"=262144
# limit 10
# '''

df = cached_query(query_string).sort_values("log_avg_epoch")
df